In [1]:
import sys
import os
import time

import numpy as np
import pandas as pd
import sklearn
import requests

from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 16})

# Let's check our software versions
print('------------')
print('### Python version: ' + __import__('sys').version)
print('### NumPy version: ' + np.__version__)
print('### Scikit-learn version: ' + sklearn.__version__)
print('------------')

def var_exists(var_name):
    return (var_name in globals() or var_name in locals())

------------
### Python version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ]
### NumPy version: 1.26.0
### Scikit-learn version: 1.2.1
------------


In [2]:
# Get tech tickers
r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/stocks')
r.status_code
df = pd.DataFrame(r.json())

techTickers = []
for i in range(463):
    techTickers.append(df.iloc[i][0])
    # if df.iloc[i][1] == 'Information Technology':
    #     techTickers.append(df.iloc[i][0])

In [3]:
# AAPL specific prediction from here down.
# Load AAPL's data

# AAPL closing price stored in dfAAPLclose
r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/stocks/AAPL/filtered_data')
dfAAPLclose = pd.DataFrame(r.json())
dfAAPLclose['date'] = pd.to_datetime(dfAAPLclose['date'])
dfAAPLclose = dfAAPLclose.sort_values('date')

# 'dfbs' stores AAPL's data from the balance sheet
aaplbsURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/balancesheets/AAPL'
r = requests.get(aaplbsURL)
dfbs = pd.DataFrame(r.json())

# 'dfcf' stores AAPL's data from cash flow statement
aaplcfURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/cashflowstatements/AAPL'
r = requests.get(aaplcfURL)
dfcf = pd.DataFrame(r.json())

# 'dfe' stores AAPL's data from earnings statement
aapleURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/earnings/AAPL'
r = requests.get(aapleURL)
dfe = pd.DataFrame(r.json())

# 'dfis' stores AAPL's data from income statement
aaplisURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/incomestatements/AAPL'
r = requests.get(aaplisURL)
dfis = pd.DataFrame(r.json())

In [4]:
df_closing_price = dfAAPLclose[['date', 'closingPrice']]

In [5]:
# https://swamplocksapi.azurewebsites.net/api/Financials/economic_data/indicators
# https://swamplocksapi.azurewebsites.net/api/Financials/ex_rates
# 
r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/economic_data/indicators')
econIndicatorsDF = pd.DataFrame(r.json())
indicatorNames = []
for i in range(len(econIndicatorsDF)):
    indicatorNames.append(econIndicatorsDF.iloc[i][0])
print(indicatorNames)
econDFs = {}
for name in indicatorNames:
    print(name)
    r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/economic_data/' + name)
    try:
        curDF = pd.DataFrame(r.json())
        curDF['date'] = pd.to_datetime(curDF['date'])
        econDFs[name] = curDF.sort_values('date')
        #econDFs[name] = curDF
    except:
        continue
print(econDFs)


['10Y-TCMR', '2Y-TCMR', '30Y-TCMR', '3M-TCMR', '5Y-TCMR', '7Y-TCMR', 'ALL_COMMODITIES', 'CPI', 'Durables', 'FFR', 'GDP', 'GDPPC', 'Inflation', 'RetailSales', 'Unemployment']
10Y-TCMR
2Y-TCMR
30Y-TCMR
3M-TCMR
5Y-TCMR
7Y-TCMR
ALL_COMMODITIES
CPI
Durables
FFR
GDP
GDPPC
Inflation
RetailSales
Unemployment
{'10Y-TCMR':     indicatorName       date  value indicator
0        10Y-TCMR 1953-04-01   2.83      None
1        10Y-TCMR 1953-05-01   3.05      None
2        10Y-TCMR 1953-06-01   3.11      None
3        10Y-TCMR 1953-07-01   2.93      None
4        10Y-TCMR 1953-08-01   2.95      None
..            ...        ...    ...       ...
858      10Y-TCMR 2024-10-01   4.10      None
859      10Y-TCMR 2024-11-01   4.36      None
860      10Y-TCMR 2024-12-01   4.39      None
861      10Y-TCMR 2025-01-01   4.63      None
862      10Y-TCMR 2025-02-01   4.45      None

[863 rows x 4 columns], '2Y-TCMR':       indicatorName       date  value indicator
0           2Y-TCMR 1976-06-01   7.26      None
1

In [6]:
econ_features = ['10Y-TCMR', '2Y-TCMR', '30Y-TCMR', '3M-TCMR', '5Y-TCMR', '7Y-TCMR', 'CPI', 'Durables', 'FFR', 'GDP', 'GDPPC', 'Inflation', 'RetailSales', 'Unemployment']

from sklearn.model_selection import train_test_split
# Ensure datetime consistency
dfe['fiscalDateEnding'] = pd.to_datetime(dfe['fiscalDateEnding'])
dfcf['fiscalDateEnding'] = pd.to_datetime(dfcf['fiscalDateEnding'])
dfis['fiscalDateEnding'] = pd.to_datetime(dfis['fiscalDateEnding'])
df_closing_price['date'] = pd.to_datetime(df_closing_price['date'])

# Create next quarter end date
dfe['nextQuarterEnd'] = dfe['fiscalDateEnding'] + pd.DateOffset(months=3)
dfcf['nextQuarterEnd'] = dfcf['fiscalDateEnding'] + pd.DateOffset(months=3)
dfis['nextQuarterEnd'] = dfis['fiscalDateEnding'] + pd.DateOffset(months=3)

# Sort by date
dfe = dfe.sort_values('nextQuarterEnd')
dfcf = dfcf.sort_values('nextQuarterEnd')
dfis = dfis.sort_values('nextQuarterEnd')
df_closing_price = df_closing_price.sort_values('date')

merged_financials = pd.merge(
    dfe[['ticker', 'fiscalDateEnding', 'nextQuarterEnd', 'reportedEPS']],
    dfcf[['ticker', 'fiscalDateEnding', 'netIncome', 'paymentsForRepurchaseOfCommonStock', 'profitLoss']],
    on=['ticker', 'fiscalDateEnding'],
    how='inner'
)

# now merge the result with dfis
merged_financials = pd.merge(
    merged_financials,
    dfis[['ticker', 'fiscalDateEnding', 'researchAndDevelopment', 'operatingExpenses', 'sellingGeneralAndAdministrative']],
    on=['ticker', 'fiscalDateEnding'],
    how='inner'
)

# Merge with closing price using merge_asof
final_data = pd.merge_asof(
    merged_financials,
    df_closing_price,
    left_on='nextQuarterEnd',
    right_on='date',
    direction='forward',
    tolerance=pd.Timedelta("30 days")
)

for econ_name in econ_features:
        if econ_name in econDFs:
            econ_df = econDFs[econ_name][['date', 'value']].copy()
            econ_df = econ_df.rename(columns={'value': econ_name})  # Rename value column to indicator name
            # Aggregate daily/monthly data to quarterly (use last value in quarter)
            econ_df['quarter_end'] = econ_df['date'] + pd.offsets.QuarterEnd(0)
            econ_df = econ_df.groupby('quarter_end').last().reset_index()[['quarter_end', econ_name]]
            print(f"{econ_name} econ_df shape: {econ_df.shape}")
            print(f"{econ_name} econ_df columns: {econ_df.columns.tolist()}")
            # Merge with final_data
            final_data = pd.merge_asof(
                final_data.sort_values('nextQuarterEnd'),
                econ_df,
                left_on='nextQuarterEnd',
                right_on='quarter_end',
                direction='backward',  # Take the most recent economic data before nextQuarterEnd
                tolerance=pd.Timedelta("90 days")
            )
existing_econ_features = [f for f in econ_features if f in final_data.columns]
if existing_econ_features:
    print(f"Imputing missing values for: {existing_econ_features}")
    # Forward fill first, then fill remaining with mean
    final_data[existing_econ_features] = final_data[existing_econ_features].fillna(method='ffill')
    final_data[existing_econ_features] = final_data[existing_econ_features].fillna(final_data[existing_econ_features].mean())
else:
    print(f"Warning: No economic features merged for {t}")
        # Drop rows with missing economic data (optional; adjust based on needs)
final_data = final_data.dropna(subset=econ_features).reset_index(drop=True)

        # Define features including economic indicators
features = [
    'reportedEPS', 'netIncome', 'paymentsForRepurchaseOfCommonStock',
    'profitLoss', 'researchAndDevelopment', 'operatingExpenses',
    'sellingGeneralAndAdministrative'
] + econ_features
X = final_data[features]
y = final_data['closingPrice']
# Verify shapes
print("X shape:", X.shape)  #
print("y shape:", y.shape)  # 

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Display shapes
print(f'Training set: {X_train.shape}, {y_train.shape}')  # (34, 5), (34,)
print(f'Validation set: {X_val.shape}, {y_val.shape}')  # (11, 5), (11,)
print(f'Test set: {X_test.shape}, {y_test.shape}')      # (12, 5), (12,)

10Y-TCMR econ_df shape: (288, 2)
10Y-TCMR econ_df columns: ['quarter_end', '10Y-TCMR']
2Y-TCMR econ_df shape: (197, 2)
2Y-TCMR econ_df columns: ['quarter_end', '2Y-TCMR']
30Y-TCMR econ_df shape: (192, 2)
30Y-TCMR econ_df columns: ['quarter_end', '30Y-TCMR']
3M-TCMR econ_df shape: (176, 2)
3M-TCMR econ_df columns: ['quarter_end', '3M-TCMR']
5Y-TCMR econ_df shape: (254, 2)
5Y-TCMR econ_df columns: ['quarter_end', '5Y-TCMR']
7Y-TCMR econ_df shape: (224, 2)
7Y-TCMR econ_df columns: ['quarter_end', '7Y-TCMR']
CPI econ_df shape: (449, 2)
CPI econ_df columns: ['quarter_end', 'CPI']
Durables econ_df shape: (133, 2)
Durables econ_df columns: ['quarter_end', 'Durables']
FFR econ_df shape: (284, 2)
FFR econ_df columns: ['quarter_end', 'FFR']
GDP econ_df shape: (92, 2)
GDP econ_df columns: ['quarter_end', 'GDP']
GDPPC econ_df shape: (312, 2)
GDPPC econ_df columns: ['quarter_end', 'GDPPC']
Inflation econ_df shape: (64, 2)
Inflation econ_df columns: ['quarter_end', 'Inflation']
RetailSales econ_df s

/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/1327847336.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price['date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/1327847336.py:56: FutureWarning: Passing 'suffixes' which cause duplicate columns {'quarter_end_x'} in the result is deprecated and will raise a MergeError in a future version.
  final_data = pd.merge_asof(
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/1327847336.py:56: FutureWarning: Passing 'suffixes' which cause duplicate columns {'quarter_end_x'} in the result is deprecated and will raise a MergeError in a future version.
  final_data = pd.merge_asof(
/var/folders/8g/

In [7]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import joblib

# Log-transform y
y_log = np.log1p(y)

# Split data
X_train, X_temp, y_train_log, y_temp_log = train_test_split(X, y_log, test_size=0.4, random_state=42)
X_val, X_test, y_val_log, y_test_log = train_test_split(X_temp, y_temp_log, test_size=0.5, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Grid search for SVR
param_grid = {
    'C': [10, 100, 1000],
    'gamma': [0.01, 0.1, 1],
    'kernel': ['rbf']
}
svm = SVR()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train_log)

# Best model
best_svm = grid_search.best_estimator_
print("Best params:", grid_search.best_params_)

# Predict and evaluate on log scale
y_val_pred_log = best_svm.predict(X_val_scaled)
val_mse_log = mean_squared_error(y_val_log, y_val_pred_log)
print(f"Validation MSE (log scale): {val_mse_log}")

y_test_pred_log = best_svm.predict(X_test_scaled)
test_mse_log = mean_squared_error(y_test_log, y_test_pred_log)
print(f"Test MSE (log scale): {test_mse_log}")

# Convert back to original scale
y_val_pred = np.expm1(y_val_pred_log)
y_test_pred = np.expm1(y_test_pred_log)
val_mse = mean_squared_error(np.expm1(y_val_log), y_val_pred)
test_mse = mean_squared_error(np.expm1(y_test_log), y_test_pred)
print(f"Validation MSE (original scale): {val_mse}")
print(f"Test MSE (original scale): {test_mse}")

ticker = "AAPL"

model_dir = os.path.join('Models', ticker)
os.makedirs(model_dir, exist_ok=True)

# save model
joblib.dump(best_svm, os.path.join(model_dir, 'best_svm_model.pkl'))

# save scalar
joblib.dump(scaler, os.path.join(model_dir, 'scaler.pkl'))

# save input
latest_data = X.iloc[-1].to_frame().T
future_quarters = 1
future_X = pd.DataFrame([latest_data.values[0]] * future_quarters, columns=features)

joblib.dump(future_X, os.path.join(model_dir, f'{ticker}_latest_features.pkl'))

Best params: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Validation MSE (log scale): 0.019021044971902015
Test MSE (log scale): 0.11804340258272061
Validation MSE (original scale): 121.33590353824887
Test MSE (original scale): 200.9787009653481


['Models/AAPL/AAPL_latest_features.pkl']

In [8]:

# Latest data for prediction (last row of X)
latest_data = X.iloc[-1].to_frame().T

# Scale and predict for next 2 quarters
future_quarters = 1
future_X = pd.DataFrame([latest_data.values[0]] * future_quarters, columns=features)
future_X_scaled = scaler.transform(future_X)
future_log_preds = best_svm.predict(future_X_scaled)
future_cp_preds = np.expm1(future_log_preds)

In [9]:
# Predicted price, last quarter's price
lastQuartersPrice = y.iloc[-1]
predictedQuartersPrice = future_cp_preds[0]
print('Projected price next quarter:' , predictedQuartersPrice , 'Last quarter\'s price:' , lastQuartersPrice)
print('Projected percent change by next quarter:' , (1 - lastQuartersPrice/predictedQuartersPrice) * 100)

Projected price next quarter: 200.92214611314367 Last quarter's price: 222.13
Projected percent change by next quarter: -10.555259485887492


In [10]:
r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/economic_data/indicators')
econIndicatorsDF = pd.DataFrame(r.json())
indicatorNames = []
for i in range(len(econIndicatorsDF)):
    indicatorNames.append(econIndicatorsDF.iloc[i][0])
print(indicatorNames)
econDFs = {}
for name in indicatorNames:
    print(name)
    r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/economic_data/' + name)
    try:
        curDF = pd.DataFrame(r.json())
        curDF['date'] = pd.to_datetime(curDF['date'])
        econDFs[name] = curDF.sort_values('date')
        #econDFs[name] = curDF
    except:
        continue
econ_features = ['10Y-TCMR', '2Y-TCMR', '30Y-TCMR', '3M-TCMR', '5Y-TCMR', '7Y-TCMR', 'CPI', 'Durables', 'FFR', 'GDP', 'GDPPC', 'Inflation', 'RetailSales', 'Unemployment']


['10Y-TCMR', '2Y-TCMR', '30Y-TCMR', '3M-TCMR', '5Y-TCMR', '7Y-TCMR', 'ALL_COMMODITIES', 'CPI', 'Durables', 'FFR', 'GDP', 'GDPPC', 'Inflation', 'RetailSales', 'Unemployment']
10Y-TCMR
2Y-TCMR
30Y-TCMR
3M-TCMR
5Y-TCMR
7Y-TCMR
ALL_COMMODITIES
CPI
Durables
FFR
GDP
GDPPC
Inflation
RetailSales
Unemployment


In [11]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
failed = []
loop = 1
if loop:
    for t in techTickers:
        try:
            print(t)
            # AAPL closing price stored in dfAAPLclose
            r = requests.get('https://swamplocksapi.azurewebsites.net/api/Financials/stocks/' + t + '/filtered_data')
            dfAAPLclose = pd.DataFrame(r.json())
            dfAAPLclose['date'] = pd.to_datetime(dfAAPLclose['date'])
            dfAAPLclose = dfAAPLclose.sort_values('date')

            # 'dfbs' stores AAPL's data from the balance sheet
            aaplbsURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/balancesheets/' + t
            r = requests.get(aaplbsURL)
            dfbs = pd.DataFrame(r.json())

            # 'dfcf' stores AAPL's data from cash flow statement
            aaplcfURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/cashflowstatements/' + t
            r = requests.get(aaplcfURL)
            dfcf = pd.DataFrame(r.json())

            # 'dfe' stores AAPL's data from earnings statement
            aapleURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/earnings/' + t
            r = requests.get(aapleURL)
            dfe = pd.DataFrame(r.json())

            # 'dfis' stores AAPL's data from income statement
            aaplisURL = 'https://swamplocksapi.azurewebsites.net/api/Financials/incomestatements/' + t
            r = requests.get(aaplisURL)
            dfis = pd.DataFrame(r.json())
            df_closing_price = dfAAPLclose[['date', 'closingPrice']]

            # Ensure datetime consistency
            dfe['fiscalDateEnding'] = pd.to_datetime(dfe['fiscalDateEnding'])
            dfcf['fiscalDateEnding'] = pd.to_datetime(dfcf['fiscalDateEnding'])
            dfis['fiscalDateEnding'] = pd.to_datetime(dfis['fiscalDateEnding'])
            df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])

            # Create next quarter end date
            dfe['nextQuarterEnd'] = dfe['fiscalDateEnding'] + pd.DateOffset(months=3)
            dfcf['nextQuarterEnd'] = dfcf['fiscalDateEnding'] + pd.DateOffset(months=3)
            dfis['nextQuarterEnd'] = dfis['fiscalDateEnding'] + pd.DateOffset(months=3)

            # Sort by date
            dfe = dfe.sort_values('nextQuarterEnd')
            dfcf = dfcf.sort_values('nextQuarterEnd')
            dfis = dfis.sort_values('nextQuarterEnd')
            df_closing_price = df_closing_price.sort_values('date')

            # Correlation analysis for feature selection
            def get_top_correlated_features(df, date_col, target_df, target_col, n=2):
                # Merge with target
                merged = pd.merge_asof(
                    df.sort_values(date_col),
                    target_df[[target_col, 'date']],
                    left_on=date_col,
                    right_on='date',
                    direction='forward',
                    tolerance=pd.Timedelta("30 days")
                )
                # Convert to numeric, drop non-numeric
                numeric_cols = merged.select_dtypes(include=[np.number]).columns
                merged = merged[numeric_cols]
                # Compute correlations
                corr = merged.corr()
                if target_col in corr:
                    # Get top n features (excluding target)
                    top_features = corr[target_col].sort_values(ascending=False)[1:n+1].index.tolist()
                    return top_features
                return []

            # Get top 2 correlated features per statement
            top_earnings = get_top_correlated_features(dfe, 'nextQuarterEnd', df_closing_price, 'closingPrice', n=2)
            top_cashflow = get_top_correlated_features(dfcf, 'nextQuarterEnd', df_closing_price, 'closingPrice', n=2)
            top_income = get_top_correlated_features(dfis, 'nextQuarterEnd', df_closing_price, 'closingPrice', n=2)

            print(f"Top earnings features for {t}: {top_earnings}")
            print(f"Top cash flow features for {t}: {top_cashflow}")
            print(f"Top income features for {t}: {top_income}")

            # Merge financial data with top features
            earnings_cols = ['ticker', 'fiscalDateEnding', 'nextQuarterEnd'] + top_earnings
            cashflow_cols = ['ticker', 'fiscalDateEnding'] + top_cashflow
            income_cols = ['ticker', 'fiscalDateEnding'] + top_income

            # Ensure columns exist
            earnings_cols = [c for c in earnings_cols if c in dfe.columns]
            cashflow_cols = [c for c in cashflow_cols if c in dfcf.columns]
            income_cols = [c for c in income_cols if c in dfis.columns]

            merged_financials = pd.merge(
                dfe[earnings_cols],
                dfcf[cashflow_cols],
                on=['ticker', 'fiscalDateEnding'],
                how='inner'
            )

            merged_financials = pd.merge(
                merged_financials,
                dfis[income_cols],
                on=['ticker', 'fiscalDateEnding'],
                how='inner'
            )

            # Merge with closing price
            final_data = pd.merge_asof(
                merged_financials,
                df_closing_price,
                left_on='nextQuarterEnd',
                right_on='date',
                direction='forward',
                tolerance=pd.Timedelta("30 days")
            )

            for econ_name in econ_features:
                if econ_name in econDFs:
                    econ_df = econDFs[econ_name][['date', 'value']].copy()
                    econ_df = econ_df.rename(columns={'value': econ_name})  # Rename value column to indicator name
                    # Aggregate daily/monthly data to quarterly (use last value in quarter)
                    econ_df['quarter_end'] = econ_df['date'] + pd.offsets.QuarterEnd(0)
                    econ_df = econ_df.groupby('quarter_end').last().reset_index()[['quarter_end', econ_name]]
                    #print(f"{econ_name} econ_df shape: {econ_df.shape}")
                    #print(f"{econ_name} econ_df columns: {econ_df.columns.tolist()}")
                    # Merge with final_data
                    final_data = pd.merge_asof(
                        final_data.sort_values('nextQuarterEnd'),
                        econ_df,
                        left_on='nextQuarterEnd',
                        right_on='quarter_end',
                        direction='backward',  # Take the most recent economic data before nextQuarterEnd
                        tolerance=pd.Timedelta("90 days")
                    )
            existing_econ_features = [f for f in econ_features if f in final_data.columns]
            if existing_econ_features:
                #print(f"Imputing missing values for: {existing_econ_features}")
                # Forward fill first, then fill remaining with mean
                final_data[existing_econ_features] = final_data[existing_econ_features].fillna(method='ffill')
                final_data[existing_econ_features] = final_data[existing_econ_features].fillna(final_data[existing_econ_features].mean())
            else:
                print(f"Warning: No economic features merged for {t}")
                    # Drop rows with missing economic data (optional; adjust based on needs)
            final_data = final_data.dropna(subset=econ_features).reset_index(drop=True)

                    # Define features including economic indicators
            # Define features (top correlated + economic)
            financial_features = top_earnings + top_cashflow + top_income
            features = financial_features + existing_econ_features
            X = final_data[features]
            y = final_data['closingPrice']

            # Verify shapes
            #print("X shape:", X.shape)  #
            #print("y shape:", y.shape)  # 

            # Split data
            X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
            X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

            # Display shapes
            # print(f'Training set: {X_train.shape}, {y_train.shape}')  # (34, 5), (34,)
            # print(f'Validation set: {X_val.shape}, {y_val.shape}')  # (11, 5), (11,)
            # print(f'Test set: {X_test.shape}, {y_test.shape}')      # (12, 5), (12,)


            # Log-transform y
            y_log = np.log1p(y)

            # Split data
            X_train, X_temp, y_train_log, y_temp_log = train_test_split(X, y_log, test_size=0.4, random_state=42)
            X_val, X_test, y_val_log, y_test_log = train_test_split(X_temp, y_temp_log, test_size=0.5, random_state=42)

            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_val_scaled = scaler.transform(X_val)
            X_test_scaled = scaler.transform(X_test)

            # Grid search for SVR
            param_grid = {
                'C': [10, 100, 1000],
                'gamma': [0.01, 0.1, 1],
                'kernel': ['rbf']
            }
            svm = SVR()
            grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='neg_mean_squared_error')
            grid_search.fit(X_train_scaled, y_train_log)

            # Best model
            best_svm = grid_search.best_estimator_
            #print("Best params:", grid_search.best_params_)

            # Predict and evaluate on log scale
            y_val_pred_log = best_svm.predict(X_val_scaled)
            val_mse_log = mean_squared_error(y_val_log, y_val_pred_log)
            #print(f"Validation MSE (log scale): {val_mse_log}")

            model_dir = os.path.join('Models', t)
            os.makedirs(model_dir, exist_ok=True)

            # save model
            joblib.dump(best_svm, os.path.join(model_dir, 'best_svm_model.pkl'))

            # save scalar
            joblib.dump(scaler, os.path.join(model_dir, 'scaler.pkl'))

            # save input
            latest_data = X.iloc[-1].to_frame().T
            future_quarters = 1
            future_X = pd.DataFrame([latest_data.values[0]] * future_quarters, columns=features)

            joblib.dump(future_X, os.path.join(model_dir, f'{t}_latest_features.pkl'))

            y_test_pred_log = best_svm.predict(X_test_scaled)
            test_mse_log = mean_squared_error(y_test_log, y_test_pred_log)
            #print(f"Test MSE (log scale): {test_mse_log}")

            # Convert back to original scale
            y_val_pred = np.expm1(y_val_pred_log)
            y_test_pred = np.expm1(y_test_pred_log)
            val_mse = mean_squared_error(np.expm1(y_val_log), y_val_pred)
            test_mse = mean_squared_error(np.expm1(y_test_log), y_test_pred)
            #print(f"Validation MSE (original scale): {val_mse}")
            #print(f"Test MSE (original scale): {test_mse}")
            # Latest data for prediction (last row of X)
            latest_data = X.iloc[-1].to_frame().T

            # Scale and predict for next 2 quarters
            future_quarters = 1
            future_X = pd.DataFrame([latest_data.values[0]] * future_quarters, columns=features)
            future_X_scaled = scaler.transform(future_X)
            future_log_preds = best_svm.predict(future_X_scaled)
            future_cp_preds = np.expm1(future_log_preds)
            #print(t, future_cp_preds)
        except Exception as e:
            failed.append(t)
            print(f"Stock {t} failed. Error {e}")

print(failed)

A


/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:45: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:133: FutureWarning: Passing 'suffixes' whi

Top earnings features for A: ['reportedEPS', 'estimatedEPS']
Top cash flow features for A: ['operatingCashFlow', 'changeInOperatingLiabilities']
Top income features for A: ['ebitda', 'incomeBeforeTax']
Stock A failed. Error Input y contains NaN.
AAPL


/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:45: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:133: FutureWarning: Passing 'suffixes' whi

Top earnings features for AAPL: ['reportedEPS', 'estimatedEPS']
Top cash flow features for AAPL: ['netIncome', 'paymentsForRepurchaseOfCommonStock']
Top income features for AAPL: ['researchAndDevelopment', 'operatingExpenses']
ABBV


...

/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:45: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:133: FutureWarning: Passing 'suffixes' whi

Top earnings features for AVY: ['reportedEPS', 'estimatedEPS']
Top cash flow features for AVY: ['dividendPayoutCommonStock', 'dividendPayout']
Top income features for AVY: ['nonInterestIncome', 'totalRevenue']
AWK


/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:45: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_closing_price.loc[:, 'date'] = pd.to_datetime(df_closing_price['date'])
/var/folders/8g/_56q3nhs07g6dmqp43hb7tqc0000gn/T/ipykernel_73483/4005159894.py:133: FutureWarning: Passing 'suffixes' whi

Top earnings features for AWK: ['reportedEPS', 'estimatedEPS']
Top cash flow features for AWK: ['dividendPayout', 'dividendPayoutCommonStock']
Top income features for AWK: ['nonInterestIncome', 'depreciationAndAmortization']
AXP


KeyboardInterrupt: 